# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. ___________________
2. __________________
3. ____________________
4. ___________________
5. ___________________

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [241]:
#Code Block : Set Initial State (Must handle dynamic inputs)
import networkx as nx

graph = nx.Graph()

def set_initial_state(edges,start):
    graph.add_weighted_edges_from(edges)
    starting_location = start
    initial_state =(starting_location, frozenset([starting_location]))
    return graph, starting_location, initial_state



In [176]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)
import numpy as np

nodes = sorted(graph.nodes())
print(nodes)
number_of_nodes = len(nodes)
print(number_of_nodes)
list_nodes = list(enumerate(nodes))
print(list_nodes)
dictionary_nodes = {node : i for i, node in list_nodes}
print(dictionary_nodes)

#Initialize transition and cost matrices
transition_matrix = np.zeros((number_of_nodes, number_of_nodes))
cost_matrix = np.zeros((number_of_nodes, number_of_nodes))

for edge in graph.edges(data=True):
    u, v, data = edge
    i, j = dictionary_nodes[u], dictionary_nodes[v]
    transition_matrix[i, j] = 1
    transition_matrix[j, i] = 1  
    cost_matrix[i, j] = data['weight']
    cost_matrix[j, i] = data['weight']
    
print(transition_matrix)
print(cost_matrix)

['Madukkur', 'Orathanadu', 'Pachiyur', 'Regunathapuram', 'Thanjavur', 'Vadakkur']
6
[(0, 'Madukkur'), (1, 'Orathanadu'), (2, 'Pachiyur'), (3, 'Regunathapuram'), (4, 'Thanjavur'), (5, 'Vadakkur')]
{'Madukkur': 0, 'Orathanadu': 1, 'Pachiyur': 2, 'Regunathapuram': 3, 'Thanjavur': 4, 'Vadakkur': 5}
[[0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 0. 0. 1.]
 [1. 1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 1. 0. 1.]
 [0. 1. 1. 0. 1. 0.]]
[[ 0.  0. 10. 11.  0.  0.]
 [ 0.  0.  8.  0.  0. 16.]
 [10.  8.  0. 12.  0.  9.]
 [11.  0. 12.  0. 17.  0.]
 [ 0.  0.  0. 17.  0. 14.]
 [ 0. 16.  9.  0. 14.  0.]]


In [178]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented

def transition_function_ucs (graph, current_state, action):
    current_vertex, visited_places = current_state
    new_vertex = action
    
    #Update the list of visited places
    new_vistited_places =  visited_places | {new_vertex} 
    
    
    # Get the cost of the action (moving along the edge)
    cost = graph[current_vertex][new_vertex]['weight']
    return (new_vertex, new_vistited_places), cost

# def transtion_function_ucs(graph):
#     for neighbor in graph.neighbors(current_vertex):
#         if neighbor not in visited_places:  # Only consider unvisited neighbors
            
#             new_state, action_cost = transition_function_ucs(graph, current_state, neighbor)
    

In [193]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

def goal_test(vistited_places,graph):
    if len(vistited_places) == graph.number_of_nodes():
            return True
    else:
        return False
    

### 2.	Definition of Algorithm 1 (Uniformed Cost Search)

In [211]:
#Code Block : Function for algorithm 1 implementation
import heapq

def uniform_cost_search(graph, start):
   
    priority_queue = []
    heapq.heappush(priority_queue, (0, initial_state, []))
    visited_states = {}
    
    #complexity tracking
    nodes_expanded_ucs = 0
    max_queue_size_ucs = 0

    while priority_queue:
        cost, current_state, path = heapq.heappop(priority_queue)
        current_vertex, visited_places = current_state
        nodes_expanded_ucs += 1
        max_queue_size_ucs = max(max_queue_size, len(priority_queue))
        
        # Goal test
        if goal_test(visited_places, graph):
            return cost, path + [current_vertex], nodes_expanded_ucs, max_queue_size_ucs
        
        # Skip processing if this state has been visited with a lower cost
        if current_state in visited_states and visited_states[current_state] <= cost:
            continue
        
        # Update the visited states with the current state's cost
        visited_states[current_state] = cost
        
        transition_function_ucs(graph)
        
        # Expand the current node
        for neighbor in graph.neighbors(current_vertex):
            if neighbor not in visited_places:  # Only consider unvisited neighbors
                new_state, action_cost = transition_function_ucs(graph, current_state, neighbor)
                heapq.heappush(priority_queue, (cost + action_cost, new_state, path + [current_vertex]))
    
    return float('inf'), []  # If no solution found


        

Total cost: 66
Path taken: Pachiyur -> Orathanadu -> Vadakkur -> Thanjavur -> Regunathapuram -> Madukkur


### 3.	Definition of Algorithm 2 (A* Search)

In [221]:
#Code Block : Function for algorithm 2 implementation

import heapq

# Heuristic function to estimate the remaining cost to visit all nodes
def calculate_heuristic(unvisited_nodes, graph, current_node):
        heuristic_cost = 0
        for node in unvisited_nodes:
            if graph.has_edge(current_node, node):
                min_cost = graph[current_node][node]['weight']
                heuristic_cost += min_cost
        return heuristic_cost


def a_star_search(graph, start):
    queue = [(0, 0, start, [start], frozenset([start]))]
    visited = {}
    total_nodes = len(graph)
    
    # Complexity tracking
    nodes_expanded = 0
    max_queue_size = 0
    
    while queue:
        current_priority, current_cost, current_node, path, visited_nodes = heapq.heappop(queue)
        
        nodes_expanded += 1
        max_queue_size = max(max_queue_size, len(queue))
        
        #Goal test
        if goal_test(visited_nodes,graph) :
            return path, current_cost, nodes_expanded, max_queue_size
        
        for neighbor, data in graph[current_node].items():
            if neighbor not in visited_nodes:
                new_visited_nodes = visited_nodes | {neighbor}
                new_cost = current_cost + data['weight']
                state = (neighbor, frozenset(new_visited_nodes))
                if state not in visited or new_cost < visited[state]:
                    visited[state] = new_cost
                    heuristic = calculate_heuristic(set(graph.nodes()) - new_visited_nodes, graph, neighbor)
                    priority = new_cost + heuristic
                    heapq.heappush(queue, (priority, new_cost, neighbor, path + [neighbor], new_visited_nodes))
    
    return None, float('inf'), nodes_expanded, max_queue_size





### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [233]:
#Code Block : Function & call to get inputs (start/end state)

#dynamic edges
input_edges = [
    ('Thanjavur', 'Vadakkur', 14),
    ('Thanjavur', 'Regunathapuram', 17),
    ('Vadakkur', 'Pachiyur', 9),
    ('Vadakkur', 'Orathanadu', 16),
    ('Orathanadu', 'Pachiyur', 8),
    ('Pachiyur', 'Regunathapuram', 12),
    ('Pachiyur', 'Madukkur', 10),
    ('Regunathapuram', 'Madukkur', 11)
]
start = 'Pachiyur'

graph,starting_location,initial_state = set_initial_state(input_edges,start)
print("graph",graph)
print("starting_location",starting_location)
print("initial_state",initial_state)



graph Graph with 6 nodes and 8 edges
starting_location Pachiyur
initial_state ('Pachiyur', frozenset({'Pachiyur'}))


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [235]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))

cost, path,nodes_expanded_ucs,max_queue_size_ucs = uniform_cost_search(graph, starting_location)
print("UCS Total cost:", total_cost)
print("UCS Path taken:", " -> ".join(path))

Total cost: 66
Path taken: Pachiyur -> Orathanadu -> Vadakkur -> Thanjavur -> Regunathapuram -> Madukkur


In [ ]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))
path, current_cost, nodes_expanded, max_queue_size = a_star_search(graph, start)

print("A* Path taken:", " -> ".join(path))
print("A* Total cost:", current_cost)

Path taken: Pachiyur -> Orathanadu -> Vadakkur -> Thanjavur -> Regunathapuram -> Madukkur
Total cost: 66


### 5.	Comparitive Analysis

In [239]:
#Code Block : Print the Time & Space complexity of algorithm 1
print("UCS Time Complexity: "nodes_expanded_ucs)
print("UCS Space Complexity: "max_queue_size_ucs)

NameError: name 'O' is not defined

In [215]:
#Code Block : Print the Time & Space complexity of algorithm 2
print("A* Time Complexity: ",nodes_expanded)
print("A* Space Complexity: ",max_queue_size)

19 4


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : _______________________________________________

________________________________________________________

_________________________________________________________